In [ ]:
import http.server
import threading
import urllib.parse
import tkinter as tk
from tkinter import scrolledtext
from twilio.rest import Client


twilio_whatsapp_number = ''  # Twilio sandbox WhatsApp numbe
account_sid = ''
auth_token = ''
client = Client(account_sid, auth_token)


# Tkinter setup for GUI
root = tk.Tk()
root.title("WhatsApp Management Solution")

# Chat Display
chat_display = scrolledtext.ScrolledText(root, width=50, height=20, state='normal')
chat_display.grid(row=0, column=0, columnspan=2, padx=10, pady=10)

# Entry to send messages
entry_message = tk.Entry(root, width=40)
entry_message.grid(row=1, column=0, padx=10, pady=10)

# Function to send WhatsApp message
def send_message():
    message_body = entry_message.get()
    if not message_body:
        return
    
    try:
        message = client.messages.create(
            from_=twilio_whatsapp_number,
            body=message_body,
            to=''  # Change this to the recipient's WhatsApp number
        )
        chat_display.insert(tk.END, f"You: {message_body}\n")
        entry_message.delete(0, tk.END)
    except Exception as e:
        chat_display.insert(tk.END, f"Error: {e}\n")

# Button to send message
send_button = tk.Button(root, text="Send Message", command=send_message)
send_button.grid(row=1, column=1, padx=10, pady=10)

# HTTP server for handling Twilio webhook
class TwilioWebhookHandler(http.server.BaseHTTPRequestHandler):
    def do_POST(self):
        # Parse form data
        content_length = int(self.headers['Content-Length'])
        post_data = self.rfile.read(content_length)
        parsed_data = urllib.parse.parse_qs(post_data.decode('utf-8'))
        
        # Extract message details
        from_number = parsed_data.get('From', [''])[0]
        body = parsed_data.get('Body', [''])[0]

        # Update the chat display in the GUI
        chat_display.insert(tk.END, f"Recipient: {body}\n")

        # Send response back to Twilio
        self.send_response(200)
        self.send_header('Content-type', 'text/plain')
        self.end_headers()
        self.wfile.write(b"Received")

# Start HTTP server in a new thread
def start_server():
    server_address = ('', 8000)
    httpd = http.server.HTTPServer(server_address, TwilioWebhookHandler)
    print("Starting server on port 8000...")
    httpd.serve_forever()

# Start the HTTP server thread
threading.Thread(target=start_server, daemon=True).start()

# Start the Tkinter GUI loop
root.mainloop()


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 30512)
Traceback (most recent call last):
  File "C:\Users\Qureshi\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "C:\Users\Qureshi\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "C:\Users\Qureshi\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\Users\Qureshi\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 755, in __init__
    self.handle()
  File "C:\Users\Qureshi\AppData\Local\Programs\Python\Python311\Lib\http\server.py", line 436, in handle
    self.handle_one_request()
  File "C:\Users\Qureshi\AppData\Local\Programs\Python\Python311\Lib\http